In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.feature_selection import *
from sklearn.preprocessing import MinMaxScaler as MMS,LabelEncoder as LE
from sklearn.decomposition import PCA
from sklearn.metrics import *
from sklearn.kernel_ridge import *
from sklearn import tree
from sklearn.ensemble import *
from sklearn.ensemble import *
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.model_selection import *
from sklearn.linear_model import *
from collections import defaultdict
from sklearn.neural_network import MLPClassifier
from encoder import Encoder
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:

def encode_data(x_train, numerical_columns, d, enc_create=True):
    cat_cols = x_train.drop(numerical_cols, axis=1).fillna('NA')

    label_enc_map = {}

    if enc_create:
        enc = ()
        cat_data = cat_cols.apply(lambda x: d[x.name].fit_transform(x))
    else:
        cat_data = cat_cols.apply(lambda x: d[x.name].transform(x))

    df = pd.concat([x_train[numerical_cols].fillna(0), cat_data], axis=1)
    return df.values


In [3]:
df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/promotion/train_LZdllcl.csv')
print df.columns
cols=df.columns
n=len(cols)


data_cols=cols[1:n-1]
label_cols=cols[n-1]
# print df
print df[label_cols].value_counts()

Index([u'employee_id', u'department', u'region', u'education', u'gender',
       u'recruitment_channel', u'no_of_trainings', u'age',
       u'previous_year_rating', u'length_of_service', u'KPIs_met >80%',
       u'awards_won?', u'avg_training_score', u'is_promoted'],
      dtype='object')
0    50140
1     4668
Name: is_promoted, dtype: int64


In [4]:

numerical_cols=['no_of_trainings','age','previous_year_rating','length_of_service',
                'KPIs_met >80%','awards_won?','avg_training_score']
d = defaultdict(LE)

X=encode_data(df[data_cols], numerical_cols, d)
Y = df[label_cols]
print len(X), len(Y)
mms=MMS()
smote = ADASYN(ratio='minority', n_neighbors= 5, n_jobs=24)
X,Y=smote.fit_sample(X,Y)
print len(X), len(Y)
X=mms.fit_transform(X)



54808 54808


100424 100424


In [7]:


"""
model = GridSearchCV(LogisticRegression(), cv =5, n_jobs=24, param_grid={'C':map(lambda x:10**(-x),range(1,5)),
                                                                         'class_weight':['balanced'],
                                                                         'penalty':['l1','l2']},


model = GridSearchCV(RandomForestClassifier(), verbose=5,cv =5, n_jobs=24, 
                     param_grid={'n_estimators':[100,1000],'class_weight':['balanced']}
                     )
                     )
"""
model = GridSearchCV(GradientBoostingClassifier(), cv = 5, n_jobs=36, verbose=5, param_grid={'n_estimators':range(100,500,200),'learning_rate':[0.1]})

model.fit(X,Y)
print 'Best',model.best_score_
print 'All', model.cv_results_


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[CV] n_estimators=100, learning_rate=0.1 .............................


[CV] n_estimators=100, learning_rate=0.1 .............................


[CV] n_estimators=100, learning_rate=0.1 .............................


[CV] n_estimators=100, learning_rate=0.1 .............................


...

[CV]  n_estimators=300, learning_rate=0.1, score=0.995817774459, total=  58.0s


[Parallel(n_jobs=36)]: Done   8 out of  10 | elapsed:   59.0s remaining:   14.7s


[CV]  n_estimators=300, learning_rate=0.1, score=0.766293253672, total=  59.5s


[Parallel(n_jobs=36)]: Done  10 out of  10 | elapsed:  1.0min finished


Best 0.950011949335
All {'std_train_score': array([ 0.01692552,  0.01606397]), 'split4_test_score': array([ 0.99004182,  0.99566819]), 'rank_test_score': array([2, 1], dtype=int32), 'split2_train_score': array([ 0.94625275,  0.95816478]), 'param_n_estimators': masked_array(data = [100 300],
             mask = [False False],
       fill_value = ?)
, 'std_score_time': array([ 0.01221443,  0.01697626]), 'split4_train_score': array([ 0.94692557,  0.95845158]), 'split2_test_score': array([ 0.99063978,  0.99681354]), 'mean_score_time': array([ 0.07879357,  0.11877117]), 'mean_fit_time': array([ 26.38520241,  58.15186863]), 'split3_train_score': array([ 0.94686267,  0.95867511]), 'split0_train_score': array([ 0.98925802,  0.99860591]), 'std_test_score': array([ 0.0915619 ,  0.09186109]), 'param_learning_rate': masked_array(data = [0.1 0.1],
             mask = [False False],
       fill_value = ?)
, 'split1_train_score': array([ 0.94780866,  0.95850085]), 'split0_test_score': array([ 0.76176

In [6]:
model.cv


In [10]:
df_test = pd.read_csv('/Users/bjayaram/Documents/Kaggle/promotion/test_2umaH9m.csv')
print df_test.columns
cols=df_test.columns
m=len(cols)


data_cols=cols[1:m]
print df

Index([u'employee_id', u'department', u'region', u'education', u'gender',
       u'recruitment_channel', u'no_of_trainings', u'age',
       u'previous_year_rating', u'length_of_service', u'KPIs_met >80%',
       u'awards_won?', u'avg_training_score'],
      dtype='object')
       employee_id         department     region         education gender  \
0            65438  Sales & Marketing   region_7  Master's & above      f   
1            65141         Operations  region_22        Bachelor's      m   
2             7513  Sales & Marketing  region_19        Bachelor's      m   
3             2542  Sales & Marketing  region_23        Bachelor's      m   
4            48945         Technology  region_26        Bachelor's      m   
5            58896          Analytics   region_2        Bachelor's      m   
6            20379         Operations  region_20        Bachelor's      f   
7            16290         Operations  region_34  Master's & above      m   
8            73202          Analy

In [11]:

# d = defaultdict(LE)

X_test=encode_data(df_test[data_cols], numerical_cols, d, enc_create=False)
# mms=MMS()
X_test=mms.transform(X_test)
print X_test


[[ 0.          0.1         0.         ...,  0.          1.          1.        ]
 [ 0.          0.275       0.6        ...,  0.          0.          0.        ]
 [ 0.          0.275       0.2        ...,  0.          1.          0.        ]
 ..., 
 [ 0.          0.15        0.8        ...,  0.          0.          1.        ]
 [ 0.22222222  0.175       0.         ...,  0.          1.          1.        ]
 [ 0.22222222  0.5         1.         ...,  0.66666667  1.          0.        ]]


In [12]:
Y_test=model.predict(X_test)
res=pd.DataFrame()
res['employee_id']=df_test['employee_id']
res['is_promoted']=Y_test
res.to_csv('/Users/bjayaram/Documents/Kaggle/promotion/results.csv', index=None)